In [1]:
! pip install firecrawl-py


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
! pip install python-dotenv


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [1]:
# Install with pip install firecrawl-py
import os
from firecrawl import FirecrawlApp

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()  # This loads the variables from .env

api_key = os.getenv('FIRECRAWL_API_KEY')
if not api_key:
    raise ValueError("No API key provided. Please set the FIRECRAWL_API_KEY environment variable.")
app = FirecrawlApp(api_key=api_key)

In [54]:
# Scrape a single URL
url = 'https://www.poppykidsdental.com/post/pediatric-dentistry-the-key-to-lifelong-healthy-smiles-for-kids'
scraped_data = app.scrape_url(url)

# # Crawl a website
# crawl_url = 'https://mendable.ai'
# params = {
#     'pageOptions': {
#         'onlyMainContent': True
#     }
# }
# crawl_result = app.crawl_url(crawl_url, params=params)

In [ ]:
print(type(scraped_data))
print(scraped_data)

# ANTHROPIC (Sonnet-3.5) - Sucks cause of output limit of 4k. Otherwise great. lol

In [23]:
import os
import json
from dotenv import load_dotenv
from anthropic import Anthropic

# Load environment variables
load_dotenv()

# Get API key from environment variable
api_key = os.getenv("ANTHROPIC_API_KEY")

# Initialize Anthropic client
client = Anthropic(api_key=api_key)



In [39]:
# Function to extract title and content
def extract_blog_info(scraped_data):
    if not isinstance(scraped_data, dict):
        raise ValueError(f"Expected a dictionary, but got {type(scraped_data)}")
    
    # Extract title
    metadata = scraped_data.get('metadata', {})
    title = metadata.get('title', "Title not found")

    # Extract content
    content = scraped_data.get('content', "Content not found")
    
    return title, content

In [47]:
# Main function to process the scraped site
def process_scraped_site(scraped_data):
    try:
        title, content = extract_blog_info(scraped_data)
    except Exception as e:
        print(f"Error extracting blog info: {e}")
        return None, None

    # Use Claude to extract the full content
    try:
        message = client.messages.create(
            model="claude-3-5-sonnet-20240620",
            max_tokens=1000,
            messages=[
                {
                    "role": "user",
                    "content": f"""Please extract and format the full content of the following blog post, 
                    including all titles, subtitles, bullet points, and paragraphs. Maintain the original 
                    structure and formatting as much as possible.

                    Title: {title}

                    Content: {content}

                    Please provide the extracted content below, preserving all headings, subheadings, 
                    bullet points, and paragraph structures:"""
                }
            ]
        )
        return title, message.content[0].text
    except Exception as e:
        print(f"Error processing with Claude: {e}")
        return title, "Error extracting content"

In [48]:
title, extracted_content = process_scraped_site(scraped_data)
if title and extracted_content:
    print(f"Title: {title}")
    print(f"Content:\n{extracted_content}")
else:
    print("Failed to process the blog post.")

Title: How to Choose between Bamboo and Traditional Toothbrushes? | Poppy Kids Pediatric Dentistry
Content:
Here is the extracted and formatted content from the blog post:

How to Choose between Bamboo and Traditional Toothbrushes?

Dr.Andrea

June 29, 2024

Brushing Up on Sustainability: Choosing Between Bamboo and Traditional Toothbrushes
-----------------------------------------------------------------------------------

As we become more environmentally aware, the choice of everyday items like toothbrushes also reflects our commitment to sustainability. While traditional plastic toothbrushes have been the standard for decades, bamboo toothbrushes have emerged as an eco-friendly alternative. However, each type comes with its own set of pros and cons. Let's delve deeper into the differences between bamboo and traditional manual toothbrushes, highlighting not just the benefits but also the potential drawbacks.

### Material and Environmental Impact:

#### Plastic Toothbrushes: A Persi

In [49]:
# Debugging: Print the structure of scraped_data
print("\nDebugging Information:")
print("Keys in scraped_data:", scraped_data.keys())
print("Type of 'metadata':", type(scraped_data.get('metadata')))
print("Keys in 'metadata':", scraped_data.get('metadata', {}).keys())


Debugging Information:
Keys in scraped_data: dict_keys(['content', 'markdown', 'metadata', 'linksOnPage'])
Type of 'metadata': <class 'dict'>
Keys in 'metadata': dict_keys(['title', 'description', 'ogTitle', 'ogDescription', 'ogImage', 'ogLocaleAlternate', 'sourceURL', 'pageStatusCode'])


### Key learning - Anthropic is NOT suitable for what I need to do. 8192 output token limit? It is not even enough to process a simple blog. With a 200K context window basically it can only do summarization. I cry for the time I spent on this. Back to OPEANAI and GPT4o-mini. 16K output should do it. 

# OPENAI

In [ ]:
#! pip install openai

In [5]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
# Initialize OpenAI client
client = OpenAI(
    api_key=api_key,
)

In [7]:
# Function to extract title and content
def extract_blog_info(scraped_data):
    if not isinstance(scraped_data, dict):
        raise ValueError(f"Expected a dictionary, but got {type(scraped_data)}")
    
    # Extract title
    metadata = scraped_data.get('metadata', {})
    title = metadata.get('title', "Title not found")

    # Extract content
    content = scraped_data.get('content', "Content not found")
    
    return title, content

In [8]:
def process_scraped_site(scraped_data):
    try:
        title, content = extract_blog_info(scraped_data)
    except Exception as e:
        print(f"Error extracting blog info: {e}")
        return None, None

    # Use OpenAI GPT-4 to extract the full content
    try:
        prompt = f"""
        Please extract and format the full content of the following blog post, excluding any image links, alt text, writer's name, time of writing, and any sections such as 'Help Shape Our Next Topic'. Additionally, remove any hyperlinks from the content. Maintain the original structure and formatting of the main content only.


        Title: {title}

        Content: {content}

        Please provide only the cleaned and formatted content below, preserving all headings, subheadings, bullet points, and paragraph structures, but excluding any extraneous information as mentioned:
        """
        
        response = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            model="gpt-4o-mini",
            max_tokens=8000
        )
        return title, response.choices[0].message.content
    except Exception as e:
        print(f"Error processing with OpenAI GPT-4: {e}")
        return title, "Error extracting content"


In [9]:
# Function to save the content to a text file
def save_to_file(title, content):
    # Format the filename
    title = title.replace(' | Poppy Kids Pediatric Dentistry', '')
    filename = title.replace(" ", "_").replace("’s", "s") + ".txt"
    filepath = f"/Users/acrobat/Documents/GitHub/extract_html/blogs/{filename}"
    
    # Write the content to the file
    with open(filepath, "w") as file:
        file.write(content)
    
    print(f"File saved as {filepath}")

In [55]:
title, extracted_content = process_scraped_site(scraped_data)
if title and extracted_content:
    print(f"Title: {title}")
    print(f"Content:\n{extracted_content}")


else:
    print("Failed to process the blog post.")

Title: Pediatric dentistry: The key to lifelong, healthy smiles for kids | Poppy Kids Pediatric Dentistry
Content:
# Pediatric dentistry: The key to lifelong, healthy smiles for kids

As a Board Certified pediatric dentist, I am deeply invested in the unique oral health journey of each child. At Poppy Kids Pediatric Dentistry, we're not just a dental practice; we're a child's stepping stone to a lifetime of healthy smiles. Our mission aligns with the American Board of Pediatric Dentistry and the American Academy of Pediatric Dentistry's view that pediatric dentists are akin to pediatricians for children's oral health, equipped with specialized training to address the nuanced dental needs of infants through adolescents, including those with special health needs.

## The distinct role of pediatric dentists

Pediatric dentistry is a realm that extends beyond the scope of general dentistry. It's a specialty honed with two years of additional training after dental school, focusing exclusive

In [56]:
save_to_file(title, extracted_content)


File saved as /Users/acrobat/Documents/GitHub/extract_html/blogs/Pediatric_dentistry:_The_key_to_lifelong,_healthy_smiles_for_kids.txt


In [20]:
# Debugging: Print the structure of scraped_data
print("\nDebugging Information:")
print("Keys in scraped_data:", scraped_data.keys())
print("Type of 'metadata':", type(scraped_data.get('metadata')))
print("Keys in 'metadata':", scraped_data.get('metadata', {}).keys())


Debugging Information:
Keys in scraped_data: dict_keys(['content', 'markdown', 'metadata', 'linksOnPage'])
Type of 'metadata': <class 'dict'>
Keys in 'metadata': dict_keys(['title', 'description', 'ogTitle', 'ogDescription', 'ogImage', 'ogLocaleAlternate', 'sourceURL', 'pageStatusCode'])
